In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Do new simulations 

In [ ]:
from simulation import run_simulation

### Option 1: Reproduce results

#key = 'fixed_complexity'
#paramfolder = 'results/{}/'.format(key)
#results = run_simulation(paramfolder, outfolder=None)

### Option 2: Add new results to existing folder 

#run_simulation(paramfolder, outfolder=paramfolder)

### Option 3: Create completely new results or add new to existing folder.

parameters = {
    'key': 'one_per_time', 
    'n_its': 10, 
    'time': 'undefined', 
    'positions': [40],
    'complexities': [4, 5, 6], 
    'anchors': [3, 6, 9], 
    'noise_sigmas': [0, 1e-7, 1e-5, 1e-3, 1e-2],
    'success_thresholds': (np.array([0, 1e-7, 1e-5, 1e-3, 1e-2])*20+1e-10).tolist(), # TODO decide what is the success
    'sampling_strategy': 'single_time'
}
outfolder = 'results/{}/'.format(parameters['key'])
run_simulation(parameters, outfolder, solver="rightInverseOfConstraints", verbose=False)

# Read and plot results

In [ ]:
from simulation import read_results, read_params
from global_variables import DIM
from plotting_tools import add_plot_decoration, generate_labels
from matplotlib.colors import LogNorm

colormap = "YlGn"
reversed_colormap = "YlGn_r"
save_figures = True

key =  'one_per_time'
resultfolder = 'results/{}/'.format(key)
results = read_results(resultfolder + 'result_')
parameters = read_params(resultfolder + 'parameters.json')
complexities = np.array(parameters['complexities'])
positions = np.array(parameters['positions'])
anchors = np.array(parameters['anchors'])
noise_sigmas = np.array(parameters['noise_sigmas'])

print("Shape of the data: ", results['successes'].shape)
    
dims = {
    'complexities' : 0,
    'anchors' : 1,
    'positions' : 2,
    'noise_sigmas' : 3,
    'measurements' : 4,
}    

# Choose which parameters to plot - the values are indexes to the previously given parameters
indices = [[]]*len(dims)
indices[dims['complexities']] = [0] #list(range(len(complexities)))
indices[dims['anchors']] = [2] #list(range(len(anchors)))
indices[dims['positions']] = [0]
indices[dims['measurements']] = list(range(positions[0]))
indices[dims['noise_sigmas']] = list(range(len(noise_sigmas)))
grid = np.meshgrid(indices[0], indices[1], indices[2], indices[3], indices[4])
title, label = generate_labels(dims, parameters, indices)
print(label)

print("NOTE: values are nan if never visited (white)")

plt.matshow(np.squeeze(results['successes'][tuple(grid)]), cmap=colormap)
plt.title('Number of successful solutions'+title)
add_plot_decoration(label, parameters)
if save_figures:
    plt.savefig(resultfolder + "/successes_"+label[0]+".pdf", bbox_inches="tight")
plt.show()

plt.matshow(np.squeeze(results['errors'][tuple(grid)]), norm=LogNorm(), cmap=reversed_colormap) 
plt.title('Errors'+title)
add_plot_decoration(label, parameters)
if save_figures:
    plt.savefig(resultfolder + "errors_"+label[0]+".pdf", bbox_inches="tight")
plt.show()

result_dimentions = np.shape(results['successes'])
bound_theory = np.zeros(result_dimentions)

for c_idx in indices[dims['complexities']]:
    for a_idx in indices[dims['anchors']]:
        for p_idx in indices[dims['positions']]:
            for noise_idx in indices[dims['noise_sigmas']]:
                for m_idx in indices[dims['measurements']]:
                    n_min = complexities[c_idx] * 2 + DIM * complexities[c_idx]
                    if parameters['sampling_strategy'] == 'single_time':
                        max_measurements = positions[p_idx]
                    else:
                        max_measurements = positions[p_idx]*anchors[a_idx]
                    if (m_idx <= max_measurements - n_min) and (positions[p_idx]>= complexities[c_idx]):
                        bound_theory[c_idx, a_idx, p_idx, noise_idx, m_idx] = 1.0
                    if m_idx > max_measurements:
                        bound_theory[c_idx, a_idx, p_idx, noise_idx, m_idx] = np.nan

print("NOTE: shows theoretical bound () and value from which zero measurements are left (white)")
plt.matshow(np.squeeze(bound_theory[tuple(grid)]), cmap=colormap)
plt.title('Theoretical bound'+title)
add_plot_decoration(label, parameters)
if save_figures:
    plt.savefig(resultfolder + "theory_"+label[0]+".pdf", bbox_inches="tight")
plt.show()

plt.matshow(np.squeeze(results['num-not-solved'][tuple(grid)]), cmap=colormap)
plt.title('Number of solver failures'+title)
add_plot_decoration(label, parameters)
if save_figures:
    plt.savefig(resultfolder + "failures_"+label[0]+".pdf", bbox_inches="tight")
plt.show()

plt.matshow(np.squeeze(results['num-not-accurate'][tuple(grid)]), cmap=colormap)
plt.title('Number of inaccurate solutions'+title)
add_plot_decoration(label, parameters)
if save_figures:
    plt.savefig(resultfolder + "inaccurate_"+label[0]+".pdf", bbox_inches="tight")
plt.show()